# 8.2 文本预处理
- **目录**
  - 8.2.1 读取数据集
  - 8.2.2 词元化
  - 8.2.3 词表
  - 8.2.4 整合所有功能

- 对于**序列数据**处理问题，我们在 8.1节中
评估了所需的统计工具和预测时面临的挑战。
- 这样的数据存在许多种形式，**文本是最常见例子之一**。
  - 例如，一篇文章可以被简单地看作是一串单词序列，甚至是一串字符序列。
- 本节中，我们将解析文本的常见预处理步骤，通常包括：
  - （1）将文本作为字符串加载到内存中。
  - （2）将字符串拆分为**词元**（如单词和字符）。
  - （3）建立一个词表，将拆分的**词元映射到数字索引**。
  - （4）将**文本转换为数字索引序列**，方便模型操作。

In [1]:
%matplotlib inline
import collections
import re
from d2l import torch as d2l

## 8.2.1 读取数据集

- 首先从H.G.Well的[时光机器](https://www.gutenberg.org/ebooks/35)中加载文本。
  - 这是一个相当小的语料库，只有30000多个单词,可供试用，
- 现实中的文档集合可能会包含数十亿个单词。
- 下面的函数**将数据集读取到由多条文本行组成的列表中**，其中每条文本行都是一个字符串。
  - 为简单起见，这里忽略了标点符号和字母大写。


In [2]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
        
    ## 将非字母的字符全部替换成空格
    ## 消除两端空格，转换成小写
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [3]:
# d2l中相关数据的下载地址，数据集保存在亚马逊云服务平台上
d2l.DATA_URL

'http://d2l-data.s3-accelerate.amazonaws.com/'

In [4]:
## re正则表达式的用法
## pattern表示非字母,逗号，句点，句号的字符替换成空（注意不是空格）
s='He  llo,w13or45ld!'
re.sub('[^A-Za-z,.。]+','',s)

'Hello,world'

## 8.2.2 词元化

- 下面的`tokenize`函数将文本行列表（`lines`）作为输入，
- 列表中的每个元素是一个文本序列（如一条文本行）。
- **每个文本序列又被拆分成一个词元列表**
  - **词元（token）**是文本的**基本单位**。
- 最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）。


In [5]:
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines] ## 将一行文字切割成单词
    elif token == 'char':
        return [list(line) for line in lines] ## list可以将字符串转换成字符列表
    else:
        print('错误：未知词元类型：' + token)
        

## tokens是一个二维list，内层list是某行文本的词元列表
tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])
    

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [6]:
## 还可以去掉空字符（不是空格字符，这是两码事）
[line.split() for line in lines[0:10] if line != '']

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells'],
 ['i'],
 ['the',
  'time',
  'traveller',
  'for',
  'so',
  'it',
  'will',
  'be',
  'convenient',
  'to',
  'speak',
  'of',
  'him'],
 ['was',
  'expounding',
  'a',
  'recondite',
  'matter',
  'to',
  'us',
  'his',
  'grey',
  'eyes',
  'shone',
  'and']]

In [7]:
# 直接使用list将字符串转换成字符列表，这种方式很常用
[list(line) for line in ['Hello,world!','你好，世界！']]

[['H', 'e', 'l', 'l', 'o', ',', 'w', 'o', 'r', 'l', 'd', '!'],
 ['你', '好', '，', '世', '界', '！']]

## 8.2.3 词表 

- 词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。
现在，让我们构建一个字典，通常也叫做**词表（vocabulary）**，
用来将字符串类型的词元映射到从$0$开始的数字索引中。
- 我们先将训练集中的所有文档合并在一起，对它们的**唯一词元**进行统计，得到的统计结果称之为**语料（corpus）**。
- 然后根据每个唯一词元的出现频率，为其分配一个数字索引。
**很少出现的词元通常被移除**，这可以降低复杂性。
- 另外，语料库中不存在或已删除的任何词元都将映射到一个特定的**未知词元“&lt;unk&gt;”**。
- 我们可以选择增加一个列表，用于保存那些被保留的词元，
例如：**填充词元（“&lt;pad&gt;”）**；
**序列开始词元（“&lt;bos&gt;”）**；
**序列结束词元（“&lt;eos&gt;”）**。
- **词表是词元与索引对应；语料就是原文档中的词元按顺序全部对应转换成索引**。

- 构造语料库的**基本步骤**总结如下:
  - **构建词表（vocabulary）**：将字符串类型的词元映射到从0开始的数字索引中。
  - **生成语料（corpus）**：合并训练集中的所有文档，统计唯一词元。
  - **为每个唯一词元分配数字索引**：根据词元的出现频率。
  - **移除低频词元**：降低模型复杂性。
  - **未知词元（"&lt;unk&gt;"）**：映射不存在或已删除的词元。
  - **保留特殊词元**：例如填充词元（“&lt;pad&gt;”）、序列开始词元（“&lt;bos&gt;”）、序列结束词元（“&lt;eos&gt;”）。
  - **词表与索引对应**：将词元转换为数字索引。
  - **语料库转换**：将原文档中的词元按顺序全部对应转换成索引。

------------
- **说明：词频在词元索引分配中有何作用？**
  - **降低计算复杂度**：根据出现频率为词元分配索引，使得高频词元具有较小的索引值，即排序靠前。在实际应用中，这有助于提高计算效率。例如，在一些压缩算法与神经网络模型中，较小的数字可以用较少的比特表示，从而降低存储和计算复杂度。
  - **提高模型泛化能力**：高频词元在语料库中更具代表性，而低频词元可能是噪声、拼写错误或非常特殊的词汇。通过根据词频分配索引并移除低频词元，可以使模型关注更具代表性的词元，提高模型在处理新数据时的泛化能力。
  - **有利于处理稀疏性问题**：在自然语言处理任务中，词表通常非常庞大，而大多数词元在给定文本中出现的频率较低。根据词频分配索引，有助于减少稀疏性问题，从而提高模型的性能。
  - **简化模型训练**：在训练词嵌入（例如 Word2Vec 或 GloVe）时，根据词频对词元进行排序并分配索引有助于在负采样或其他优化策略中利用词频信息。例如，可以通过将词频转换为概率分布，从而更容易地从词汇表中采样词元。
---

In [8]:
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        ## 初始化tokens和reserved_tokens两个变量为list对象
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        ## 按照items的第二个元素排序，即字典的值，也就是按词频由高到低排序
        ## reverse=True表示由高到底
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        ## 保存在reserved_tokens的第一个元素，因此其索引为0        
        self.idx_to_token = ['<unk>'] + reserved_tokens
        
        ## 初始化token_to_idx，是一个字典类型数据：
        ## "词元:索引"键值对
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        
        ## 下面正式开始为idx_to_token和token_to_idx赋值
        for token, freq in self._token_freqs:
            ## 设定词频阈值，低于该阈值就忽略这个词元，不保存在字典中
            if freq < min_freq:
                break
            '''
            构造词表的关键步骤：
            （1）idx_to_token本身就是一个list，直接将词元添加进去，自动和索引按顺序对应上了，
            注意索引是从0开始，而不是从1开始。
            （2）token_to_idx则不同，它是一个字典类型，键是词元，值是词元索引，由于是依次存入idx_to_token的，
            因此直接取出idx_to_token的长度减1，即是该词元的索引。
            注意减1是因为list的索引是从0开始的，因此最后一个词元的索引是idx_to_token的长度减1。           
            '''
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    '''
    覆盖该函数，本类的对象可以通过[]操作符取值或切片
    按照后面的操作，返回指定词元的索引。  
    
    下属代码初看让人十分困惑，但是仔细看就明白了：
    （1）if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        这两行代码首先判定tokens是不是list或tuple对象，如果不是list或tuple，那么就是单个token，然后从token_to_idx
        以该token作为键取其值，即该词元的索引，然后返回给调用者，下面还有一个return就不执行了。
    （2）return [self.__getitem__(token) for token in tokens]
         这一行实际上就是获取类型为list或tuple的tokens里的单个token，然后再调用__getitem__函数，
         那么就会执行判断，然后再执行self.token_to_idx.get(tokens, self.unk)。
    （3）其实就是确保执行或最后执行的是return self.token_to_idx.get(tokens, self.unk)这一行代码。
    '''
    def __getitem__(self, tokens):
        ## 确保tokens是单个的词元，然后通过词元获取相应索引
        ## 如果词元不存在，返回unk的索引0。注意get的用法        
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk) ## 
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        ## 确保indeces是单个索引，然后通过索引获取相应词元
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    
    ## 返回<unk>词元的索引，即0
    ## 在__getitem__函数里调用
    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs
'''
计算所有词元的词频，返回的是一个字典：
键是词元，值是词频。
'''
def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

------------

- **说明：Vocab类功能详解**

In [9]:
## 计算每个token的个数
## Counter本质上就是一个dict类型
t = [t for line in lines[0:10] for t in line.split()]
tf = collections.Counter(t)
tf

Counter({'the': 2,
         'time': 2,
         'to': 2,
         'machine': 1,
         'by': 1,
         'h': 1,
         'g': 1,
         'wells': 1,
         'i': 1,
         'traveller': 1,
         'for': 1,
         'so': 1,
         'it': 1,
         'will': 1,
         'be': 1,
         'convenient': 1,
         'speak': 1,
         'of': 1,
         'him': 1,
         'was': 1,
         'expounding': 1,
         'a': 1,
         'recondite': 1,
         'matter': 1,
         'us': 1,
         'his': 1,
         'grey': 1,
         'eyes': 1,
         'shone': 1,
         'and': 1})

In [10]:
'''
[t for line in lines[0:10] for t in line.split()]改成普通嵌套for循环.
for line in lines[0:10]是外层循环；for t in line.split()是内层循环
即前面for循环是外层循环，后面for循环是内层循环。
'''
t = []
for line in lines[0:10]:
    for token in line.split():
        t.append(token)
tf = collections.Counter(t)
tf

Counter({'the': 2,
         'time': 2,
         'to': 2,
         'machine': 1,
         'by': 1,
         'h': 1,
         'g': 1,
         'wells': 1,
         'i': 1,
         'traveller': 1,
         'for': 1,
         'so': 1,
         'it': 1,
         'will': 1,
         'be': 1,
         'convenient': 1,
         'speak': 1,
         'of': 1,
         'him': 1,
         'was': 1,
         'expounding': 1,
         'a': 1,
         'recondite': 1,
         'matter': 1,
         'us': 1,
         'his': 1,
         'grey': 1,
         'eyes': 1,
         'shone': 1,
         'and': 1})

In [11]:
t

['the',
 'time',
 'machine',
 'by',
 'h',
 'g',
 'wells',
 'i',
 'the',
 'time',
 'traveller',
 'for',
 'so',
 'it',
 'will',
 'be',
 'convenient',
 'to',
 'speak',
 'of',
 'him',
 'was',
 'expounding',
 'a',
 'recondite',
 'matter',
 'to',
 'us',
 'his',
 'grey',
 'eyes',
 'shone',
 'and']

In [12]:
'''
注意sorted排序函数后两个参数的用法。
sorted函数是非就地排序，即不改变tf本身的排序，而是返回一个排序后的对象。
key参数指定排序的依据，此处是依据元组的第二个元素即词频作为排序依据。
reverse=True表示根据词频从高到低对元组进行排序。
'''
sorted(tf.items(), key=lambda x:x[1],reverse=True)

[('the', 2),
 ('time', 2),
 ('to', 2),
 ('machine', 1),
 ('by', 1),
 ('h', 1),
 ('g', 1),
 ('wells', 1),
 ('i', 1),
 ('traveller', 1),
 ('for', 1),
 ('so', 1),
 ('it', 1),
 ('will', 1),
 ('be', 1),
 ('convenient', 1),
 ('speak', 1),
 ('of', 1),
 ('him', 1),
 ('was', 1),
 ('expounding', 1),
 ('a', 1),
 ('recondite', 1),
 ('matter', 1),
 ('us', 1),
 ('his', 1),
 ('grey', 1),
 ('eyes', 1),
 ('shone', 1),
 ('and', 1)]

In [13]:
## tf是一个字典
tf.items()

dict_items([('the', 2), ('time', 2), ('machine', 1), ('by', 1), ('h', 1), ('g', 1), ('wells', 1), ('i', 1), ('traveller', 1), ('for', 1), ('so', 1), ('it', 1), ('will', 1), ('be', 1), ('convenient', 1), ('to', 2), ('speak', 1), ('of', 1), ('him', 1), ('was', 1), ('expounding', 1), ('a', 1), ('recondite', 1), ('matter', 1), ('us', 1), ('his', 1), ('grey', 1), ('eyes', 1), ('shone', 1), ('and', 1)])

In [14]:
'''
items函数返回字典的键值对，是以元组形式保存，前键后值，然后将元组保存到一个list里。
这样的确便于迭代，尤其是通过lambda函数操作。
还不是一个纯list，而是封装在dict_items里的list，当然也可以直接转换成list使用。
'''
list(tf.items())

[('the', 2),
 ('time', 2),
 ('machine', 1),
 ('by', 1),
 ('h', 1),
 ('g', 1),
 ('wells', 1),
 ('i', 1),
 ('traveller', 1),
 ('for', 1),
 ('so', 1),
 ('it', 1),
 ('will', 1),
 ('be', 1),
 ('convenient', 1),
 ('to', 2),
 ('speak', 1),
 ('of', 1),
 ('him', 1),
 ('was', 1),
 ('expounding', 1),
 ('a', 1),
 ('recondite', 1),
 ('matter', 1),
 ('us', 1),
 ('his', 1),
 ('grey', 1),
 ('eyes', 1),
 ('shone', 1),
 ('and', 1)]

In [15]:
## __getitem__函数的用法，既可切片也可进行字典操作
class A: #@save
    def __init__(self):
        self.l={'name':'zhangsan','age':21}
    
    def __len__(self):
        return len(self._l)
    
    def __getitem__(self,k):
        #print('get item')
        return self.l[k]
    
a=A()
a['name']

'zhangsan'

In [16]:
## 字典数据结构的get方法应用
dictionary = {'name': 'Alice', 'age': 25, 'city': 'Beijing'}

# 获取键'name'对应的值
name_value = dictionary.get('name')
print(name_value)  # 输出：Alice

# 获取键'gender'对应的值，此键不存在，返回默认值'Unknown'
gender_value = dictionary.get('gender', 'Unknown')
print(gender_value)  # 输出：Unknown

Alice
Unknown


----------------

- 首先使用时光机器数据集作为语料库来**构建词表**。
- 然后打印前几个高频词元及其索引。


In [17]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


- 现在可以**将每一条文本行转换成一个数字索引列表**。


In [18]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


-----------

- **说明：继续解析Vocab类**

In [19]:
## Vocab对象既可传递单个词元，又可以传递词元列表或元组，以便获取词元索引
## []功能通过覆盖__getitem__方法实现
vocab.__getitem__('and'),vocab['and'],vocab[['and','face']],vocab[('was','flushed')]

(3, 3, [3, 113], [7, 1421])

In [20]:
## 词频，由高到低排列
vocab.token_freqs[0:10]

[('the', 2261),
 ('i', 1267),
 ('and', 1245),
 ('of', 1155),
 ('a', 816),
 ('to', 695),
 ('was', 552),
 ('in', 541),
 ('that', 443),
 ('my', 440)]

In [21]:
vocab.to_tokens(0)

'<unk>'

-------

## 8.2.4 整合所有功能

- 在使用上述函数时，**将所有功能打包到`load_corpus_time_machine`函数**。
  - 该函数返回`corpus`（词元索引列表，即语料库）和`vocab`（时光机器语料库的词表）。
- 此处的改变是：
  - 为了简化后面章节中的训练，我们使用字符（而不是单词）实现文本词元化。
    - **注：** 本视频课程将之设置为单词进行词元化。
  - 时光机器数据集中的每个**文本行**不一定是一个句子或一个段落，还可能是一个单词，因此返回的`corpus`仅处理为单个列表，而不是使用多词元列表构成的一个列表。
    - **注：** 即corpus是一维列表，而不是二维列表，即不是列表的列表。


In [27]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    ## 原代码的第二个参数是'char'
    tokens = tokenize(lines, 'char')
    #tokens = tokenize(lines, 'word')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中，即一维列表，而不是列表的列表
    corpus = [vocab[token] for line in tokens for token in line]
    ## 截取前10000个词元索引作为语料库
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

## corpus将文本中的词元转换成词表中的该词元所对应的索引
corpus, vocab = load_corpus_time_machine(10000)
len(corpus), len(vocab)

(10000, 28)

In [29]:
vocab.token_to_idx

{'<unk>': 0,
 ' ': 1,
 'e': 2,
 't': 3,
 'a': 4,
 'i': 5,
 'n': 6,
 'o': 7,
 's': 8,
 'h': 9,
 'r': 10,
 'd': 11,
 'l': 12,
 'm': 13,
 'u': 14,
 'c': 15,
 'f': 16,
 'w': 17,
 'g': 18,
 'y': 19,
 'p': 20,
 'b': 21,
 'v': 22,
 'k': 23,
 'x': 24,
 'z': 25,
 'j': 26,
 'q': 27}

---------

- **说明：语料库和词表的简单用法**

In [23]:
## 比如下面代码反过来将语料中的索引转换成对应的词元，就是一个完整的文本
for i in corpus[0:10]:
    print(vocab.idx_to_token[i])


the
time
machine
by
h
g
wells
i
the
time


In [24]:
## 语料库的前10个词元索引
corpus[0:10]

[1, 19, 50, 40, 2183, 2184, 400, 2, 1, 19]

-------

## 小结

* 文本是序列数据的一种最常见的形式之一。
* 为了对文本进行预处理，我们通常将文本拆分为词元，构建词表将词元字符串映射为数字索引，并将文本数据转换为词元索引以供模型操作。